In [1]:
import pandas as pd

In [333]:
casos = pd.DataFrame({'fecha': ['2017-12-14', '2016-03-12', '2017-11-12', '2016-04-02', '2018-10-03', '2017-10-02', '2018-01-02', '2018-03-02', '2018-02-02', '2018-12-02', '2017-04-04', '2018-01-01', '2017-11-02'],
                        'id_caso': [0,1,2,3,4,5,6,7,8,9,10,11,12],
                        'descripcion': ['C0', 'C1', 'C2', 'C3','c4','5','6','7','8','9','10','11','12'],
                        'estado_caso': [1, 2, 1,2,1,1,1,2,1,2,3,1,1],
                        'categoria': ['A', 'D', 'C', 'D','A','C','A','D','D','A','C','A','C'],
                        'latitud': ['D0', 'D1', 'D2', 'D3','D0', 'D1', 'D2', 'D3','D0', 'D1', 'D2', 'D3','a'],
                        'longitud': ['D0', 'D1', 'D2', 'D3','D0', 'D1', 'D2', 'D3','D0', 'D1', 'D2', 'D3','s']},
                        index=range(0,13))

In [334]:
vatman= pd.DataFrame({'id_caso': [0,1,2,3,4,5,6,7,8,9,10,11,12],
                     'respuesta':[1,0,1,1,0,1,0,0,0,1,1,0,1]})

In [335]:
casos.head(3)

,categoria,descripcion,estado_caso,fecha,id_caso,latitud,longitud
0,A,C0,1,2017-12-14,0,D0,D0
1,D,C1,2,2016-03-12,1,D1,D1
2,C,C2,1,2017-11-12,2,D2,D2


In [336]:
vatman.head(3)

,id_caso,respuesta
0,0,1
1,1,0
2,2,1


In [337]:
casos = casos[['fecha','id_caso','categoria','estado_caso']]

In [338]:
casos = casos[(casos['fecha']>'2017-08-00')&(casos['fecha']<'2018-04-00')]

In [339]:
casos['fecha'] = casos.apply(lambda x: x['fecha'].split('-')[0],axis=1)

In [340]:
casos

,fecha,id_caso,categoria,estado_caso
0,2017,0,A,1
2,2017,2,C,1
5,2017,5,C,1
6,2018,6,A,1
7,2018,7,D,2
8,2018,8,D,1
11,2018,11,A,1
12,2017,12,C,1


In [341]:
casos['trimestre']= ['actual' for x in casos['fecha']]

In [342]:
casos

,fecha,id_caso,categoria,estado_caso,trimestre
0,2017,0,A,1,actual
2,2017,2,C,1,actual
5,2017,5,C,1,actual
6,2018,6,A,1,actual
7,2018,7,D,2,actual
8,2018,8,D,1,actual
11,2018,11,A,1,actual
12,2017,12,C,1,actual


In [343]:
casos['trimestre']= casos.apply(lambda x: 'actual' if x['fecha']=='2018' else 'anterior', axis=1)

In [344]:
casos


,fecha,id_caso,categoria,estado_caso,trimestre
0,2017,0,A,1,anterior
2,2017,2,C,1,anterior
5,2017,5,C,1,anterior
6,2018,6,A,1,actual
7,2018,7,D,2,actual
8,2018,8,D,1,actual
11,2018,11,A,1,actual
12,2017,12,C,1,anterior


In [228]:
import numpy as np

In [345]:
casos['tasa'] = np.ones(casos['fecha'].count())

In [346]:
casosAgrupados = casos.groupby(by=['categoria', 'trimestre']).apply(lambda x: (x[x['estado_caso']==1].count()/x['estado_caso'].count()))

In [347]:
casosAgrupados

fecha  id_caso  categoria  estado_caso  trimestre  tasa
categoria trimestre                                                         
A         actual       1.0      1.0        1.0          1.0        1.0   1.0
          anterior     1.0      1.0        1.0          1.0        1.0   1.0
C         anterior     1.0      1.0        1.0          1.0        1.0   1.0
D         actual       0.5      0.5        0.5          0.5        0.5   0.5

In [348]:
casosAgrupados=casosAgrupados.unstack()

In [349]:
casosAgrupados

fecha          id_caso          categoria          estado_caso  \
trimestre actual anterior  actual anterior    actual anterior      actual   
categoria                                                                   
A            1.0      1.0     1.0      1.0       1.0      1.0         1.0   
C            NaN      1.0     NaN      1.0       NaN      1.0         NaN   
D            0.5      NaN     0.5      NaN       0.5      NaN         0.5   

                   trimestre            tasa           
trimestre anterior    actual anterior actual anterior  
categoria                                              
A              1.0       1.0      1.0    1.0      1.0  
C              1.0       NaN      1.0    NaN      1.0  
D              NaN       0.5      NaN    0.5      NaN

In [350]:
tasas =casosAgrupados['tasa']

In [351]:
tasas

trimestre,actual,anterior
categoria,,
A,1.0,1.0
C,NaN,1.0
D,0.5,NaN


In [352]:
tasas = tasas.fillna(0)

In [353]:
cate = tasas[tasas['actual']>(tasas['anterior']+tasas['anterior']*0.1)]

In [354]:
cate

trimestre,actual,anterior
categoria,,
D,0.5,0.0


In [309]:
casos = casos[(casos['categoria']=='A')|(casos['categoria']=='D')]

In [310]:
casos

,categoria,descripcion,estado_caso,fecha,id_caso,latitud,longitud
0,A,C0,1,2017-12-14,0,D0,D0
1,D,C1,2,2016-03-12,1,D1,D1
3,D,C3,2,2016-04-02,3,D3,D3
4,A,c4,1,2018-10-03,4,D0,D0
6,A,6,1,2018-01-02,6,D2,D2
7,D,7,2,2018-03-02,7,D3,D3
8,D,8,1,2018-02-02,8,D0,D0
9,A,9,2,2018-12-02,9,D1,D1
11,A,11,1,2018-01-01,11,D3,D3


In [311]:
casos=casos[['categoria','id_caso']]

In [312]:
casos

,categoria,id_caso
0,A,0
1,D,1
3,D,3
4,A,4
6,A,6
7,D,7
8,D,8
9,A,9
11,A,11


In [313]:
casosVatman= pd.merge(casos,vatman,on='id_caso',how='left')

In [314]:
casosVatman

,categoria,id_caso,respuesta
0,A,0,1
1,D,1,0
2,D,3,1
3,A,4,0
4,A,6,0
5,D,7,0
6,D,8,0
7,A,9,1
8,A,11,0


In [322]:
casosVatman['tasa']=np.ones(len(casosVatman['id_caso']))

In [323]:
casosVatman

,categoria,id_caso,respuesta,tasa
0,A,0,1,1.0
1,D,1,0,1.0
2,D,3,1,1.0
3,A,4,0,1.0
4,A,6,0,1.0
5,D,7,0,1.0
6,D,8,0,1.0
7,A,9,1,1.0
8,A,11,0,1.0


In [331]:
cate = casosVatman.groupby('categoria').apply(lambda x: x[x['respuesta']==1].count()/x.count())

In [329]:
cateTasa= cate[['tasa']]

In [332]:
cate

,categoria,id_caso,respuesta,tasa
categoria,,,,
A,0.40,0.40,0.40,0.40
D,0.25,0.25,0.25,0.25


In [330]:
cateTasa

,tasa
categoria,
A,0.40
D,0.25
